In [3]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [4]:
!pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [6]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
%cd C:\Users\2910p\Desktop\DESKTOP

C:\Users\2910p\Desktop\DESKTOP


C:\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [10]:
# Make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data\\positive'

In [11]:
# http://vis-www.cs.umass.edu/lfw/

In [23]:
# Uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [24]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [13]:
# Import uuid library to generate unique image names
import uuid

In [14]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

'data\\anchor\\f10982d8-4f82-11ef-a53f-346f24c8a482.jpg'

In [15]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [17]:
def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        # img = tf.image.stateless_random_crop(img, size=(20,20,3), seed=(1,2))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
            
        data.append(img)
    
    return data

In [18]:
import os
import uuid

In [19]:
for file_name in os.listdir(os.path.join(ANC_PATH)):
    img_path = os.path.join(ANC_PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [20]:
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [22]:
import os
import random
import shutil

# Path to the folder containing images
folder_path = ANC_PATH

# Number of images to keep
num_to_keep = 1000

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out only the image files (you can customize this to match your image types)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# Check if there are more than the specified number of images
if len(image_files) > num_to_keep:
    # Randomly select images to keep
    files_to_keep = random.sample(image_files, num_to_keep)
    files_to_delete = set(image_files) - set(files_to_keep)

    # Prompt the user for confirmation before deleting files
    print(f"Number of images to delete: {len(files_to_delete)}")
    confirm = input("Are you sure you want to delete these files? (yes/no): ")

    if confirm.lower() == 'yes':
        # Delete the files that are not in the list to keep
        for file in files_to_delete:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        print("Deletion completed.")
    else:
        print("Deletion aborted.")
else:
    print(f"There are only {len(image_files)} images, which is less than or equal to the number to keep.")

Number of images to delete: 4000


Are you sure you want to delete these files? (yes/no):  Yes


Deleted: data\anchor\a06ace66-4f83-11ef-a8b9-346f24c8a482.jpg
Deleted: data\anchor\8cc58cb0-4f83-11ef-83be-346f24c8a482.jpg
Deleted: data\anchor\9687f554-4f83-11ef-8115-346f24c8a482.jpg
Deleted: data\anchor\9c897784-4f83-11ef-8351-346f24c8a482.jpg
Deleted: data\anchor\9df97f7b-4f83-11ef-b4d1-346f24c8a482.jpg
Deleted: data\anchor\0db12988-4f83-11ef-9355-346f24c8a482.jpg
Deleted: data\anchor\9394bc11-4f83-11ef-aeaf-346f24c8a482.jpg
Deleted: data\anchor\8a6ace1b-4f83-11ef-9e1f-346f24c8a482.jpg
Deleted: data\anchor\9eebd4b4-4f83-11ef-ad88-346f24c8a482.jpg
Deleted: data\anchor\a3c460c7-4f83-11ef-a489-346f24c8a482.jpg
Deleted: data\anchor\39becc85-4f83-11ef-8ac7-346f24c8a482.jpg
Deleted: data\anchor\8f36a5ef-4f83-11ef-a03b-346f24c8a482.jpg
Deleted: data\anchor\9614f619-4f83-11ef-aa79-346f24c8a482.jpg
Deleted: data\anchor\922efc23-4f83-11ef-bcf7-346f24c8a482.jpg
Deleted: data\anchor\9f8772d6-4f83-11ef-838b-346f24c8a482.jpg
Deleted: data\anchor\0d19320c-4f83-11ef-a646-346f24c8a482.jpg
Deleted:

In [23]:
import os
import random
import shutil

# Path to the folder containing images
folder_path = POS_PATH

# Number of images to keep
num_to_keep = 1000

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out only the image files (you can customize this to match your image types)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# Check if there are more than the specified number of images
if len(image_files) > num_to_keep:
    # Randomly select images to keep
    files_to_keep = random.sample(image_files, num_to_keep)
    files_to_delete = set(image_files) - set(files_to_keep)

    # Prompt the user for confirmation before deleting files
    print(f"Number of images to delete: {len(files_to_delete)}")
    confirm = input("Are you sure you want to delete these files? (yes/no): ")

    if confirm.lower() == 'yes':
        # Delete the files that are not in the list to keep
        for file in files_to_delete:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        print("Deletion completed.")
    else:
        print("Deletion aborted.")
else:
    print(f"There are only {len(image_files)} images, which is less than or equal to the number to keep.")

Number of images to delete: 4000


Are you sure you want to delete these files? (yes/no):  YES


Deleted: data\positive\d80698e3-4f7e-11ef-9603-346f24c8a482.jpg
Deleted: data\positive\c6af5ffd-4f83-11ef-ac55-346f24c8a482.jpg
Deleted: data\positive\8987f43d-4f7e-11ef-a98d-346f24c8a482.jpg
Deleted: data\positive\c31f369a-4f83-11ef-a071-346f24c8a482.jpg
Deleted: data\positive\c4165ab8-4f83-11ef-903c-346f24c8a482.jpg
Deleted: data\positive\c63f8a16-4f83-11ef-bb75-346f24c8a482.jpg
Deleted: data\positive\be6c0ea7-4f83-11ef-94b0-346f24c8a482.jpg
Deleted: data\positive\c4f01745-4f83-11ef-b3ca-346f24c8a482.jpg
Deleted: data\positive\c31e301f-4f83-11ef-93e4-346f24c8a482.jpg
Deleted: data\positive\c82db298-4f83-11ef-9a4b-346f24c8a482.jpg
Deleted: data\positive\c78d5cba-4f83-11ef-b01f-346f24c8a482.jpg
Deleted: data\positive\be59f4af-4f83-11ef-b281-346f24c8a482.jpg
Deleted: data\positive\be4bbd63-4f83-11ef-a571-346f24c8a482.jpg
Deleted: data\positive\bef22c7c-4f83-11ef-b72f-346f24c8a482.jpg
Deleted: data\positive\bcd2e143-4f83-11ef-ab66-346f24c8a482.jpg
Deleted: data\positive\c9d8b0e5-4f83-11e

In [24]:
import os
import random
import shutil

# Path to the folder containing images
folder_path = NEG_PATH

# Number of images to keep
num_to_keep = 1000

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out only the image files (you can customize this to match your image types)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# Check if there are more than the specified number of images
if len(image_files) > num_to_keep:
    # Randomly select images to keep
    files_to_keep = random.sample(image_files, num_to_keep)
    files_to_delete = set(image_files) - set(files_to_keep)

    # Prompt the user for confirmation before deleting files
    print(f"Number of images to delete: {len(files_to_delete)}")
    confirm = input("Are you sure you want to delete these files? (yes/no): ")

    if confirm.lower() == 'yes':
        # Delete the files that are not in the list to keep
        for file in files_to_delete:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        print("Deletion completed.")
    else:
        print("Deletion aborted.")
else:
    print(f"There are only {len(image_files)} images, which is less than or equal to the number to keep.")

Number of images to delete: 12233


Are you sure you want to delete these files? (yes/no):  YES


Deleted: data\negative\Atal_Bihari_Vajpayee_0010.jpg
Deleted: data\negative\Nicole_Kidman_0018.jpg
Deleted: data\negative\George_W_Bush_0353.jpg
Deleted: data\negative\Bobby_Goldwater_0001.jpg
Deleted: data\negative\John_Negroponte_0026.jpg
Deleted: data\negative\Junichiro_Koizumi_0008.jpg
Deleted: data\negative\Matthew_McConaughey_0001.jpg
Deleted: data\negative\Edward_Lu_0003.jpg
Deleted: data\negative\John_Howard_0019.jpg
Deleted: data\negative\Guillermo_Canas_0001.jpg
Deleted: data\negative\Lionel_Richie_0001.jpg
Deleted: data\negative\Tiffany_Limos_0001.jpg
Deleted: data\negative\Kurt_Warner_0002.jpg
Deleted: data\negative\Chung_Mong-joon_0001.jpg
Deleted: data\negative\Hamid_Karzai_0015.jpg
Deleted: data\negative\Stefan_Tafrov_0001.jpg
Deleted: data\negative\Dwayne_Johnson_0001.jpg
Deleted: data\negative\Hiroyuki_Yoshino_0001.jpg
Deleted: data\negative\Tommy_Franks_0002.jpg
Deleted: data\negative\Gerhard_Schroeder_0030.jpg
Deleted: data\negative\Sean_Penn_0002.jpg
Deleted: data\n